In [2]:
import os, math, subprocess
import datetime as dt

#data wrangling
import numpy as np
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

#excel library
import xlrd

In [3]:
#Loading data
df_anz = pd.read_excel('ANZ synthesised transaction dataset.xlsx', sheet_name= 'DSynth_Output_100c_3m_v3')
df_anz = df_anz.astype({"card_present_flag":'bool'})
df_anz.head()

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,True,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,False,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,...,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit
2,authorized,True,NaN,ACC-1222300524,AUD,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,...,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,151.21 -33.87,debit
3,authorized,True,NaN,ACC-1037050564,AUD,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,...,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,153.05 -26.68,debit
4,authorized,True,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,...,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,153.44 -28.06,debit


Check missing data

In [4]:
df_anz.isnull().sum()

status                   0
card_present_flag        0
bpay_biller_code     11158
account                  0
currency                 0
long_lat                 0
txn_description          0
merchant_id           4326
merchant_code        11160
first_name               0
balance                  0
date                     0
gender                   0
age                      0
merchant_suburb       4326
merchant_state        4326
extraction               0
amount                   0
transaction_id           0
country                  0
customer_id              0
merchant_long_lat     4326
movement                 0
dtype: int64

Check data type and values

In [5]:
df_anz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12043 entries, 0 to 12042
Data columns (total 23 columns):
status               12043 non-null object
card_present_flag    12043 non-null bool
bpay_biller_code     885 non-null object
account              12043 non-null object
currency             12043 non-null object
long_lat             12043 non-null object
txn_description      12043 non-null object
merchant_id          7717 non-null object
merchant_code        883 non-null float64
first_name           12043 non-null object
balance              12043 non-null float64
date                 12043 non-null datetime64[ns]
gender               12043 non-null object
age                  12043 non-null int64
merchant_suburb      7717 non-null object
merchant_state       7717 non-null object
extraction           12043 non-null object
amount               12043 non-null float64
transaction_id       12043 non-null object
country              12043 non-null object
customer_id          12043 non

status

In [6]:
df_anz.status.unique()

array(['authorized', 'posted'], dtype=object)

In [7]:
df_anz.groupby(['status'])['account'].agg(['count'])

,count
status,
authorized,7717
posted,4326


card_present_flag

In [8]:
df_anz.groupby(['card_present_flag'])['account'].agg(['count'])

,count
card_present_flag,
False,1523
True,10520


bpay_biller_code

In [9]:
df_anz.groupby(['bpay_biller_code'])['account'].agg(['count'])
#There are 883 customers have 0 as bpay_biller_code

,count
bpay_biller_code,
0,883
LAND WATER & PLANNING East Melbourne,1
THE DISCOUNT CHEMIST GROUP,1


account

In [10]:
df_anz.groupby(['account','first_name'])['status'].agg(['count']).sort_values(by='count',ascending=0).head()
#there are 100 accounts = 100 customers transactions in last 3 months

,,count
account,first_name,
ACC-1598451071,Diana,578
ACC-1222300524,Michael,303
ACC-182446574,Tonya,292
ACC-4258502723,Kimberly,260
ACC-1037050564,Rhonda,259


currency

In [11]:
df_anz.currency.unique()

array(['AUD'], dtype=object)

long_lat

In [12]:
df_anz.groupby(['long_lat','first_name'])['status'].agg(['count']).sort_values(by='count',ascending=0).head()
#there are 100 long_lat = 100 customers transactions in last 3 months

,,count
long_lat,first_name,
153.41 -27.95,Diana,578
151.23 -33.94,Michael,303
116.06 -32.00,Tonya,292
145.45 -37.74,Kimberly,260
153.10 -27.66,Rhonda,259


txn_description     

In [13]:
df_anz.txn_description.unique()

array(['POS', 'SALES-POS', 'PAYMENT', 'INTER BANK', 'PAY/SALARY',
       'PHONE BANK'], dtype=object)

In [14]:
df_anz.groupby(['txn_description'])['status'].agg(['count']).sort_values(by='count',ascending=0)
#Is POS = SALES-POS, PAYMENT = PAY/SALARY? INTERBANK = PHONE BANK?

,count
txn_description,
SALES-POS,3934
POS,3783
PAYMENT,2600
PAY/SALARY,883
INTER BANK,742
PHONE BANK,101


merchant_id     

In [15]:
df_anz.groupby(['merchant_id'])['status'].agg(['count']).sort_values(by='count',ascending=0).head()
#There are 5725 merchant_id.

,count
merchant_id,
106e1272-44ab-4dcb-a438-dd98e0071e51,14
6c84c219-96ea-4c6f-9de5-0a28bcaec62f,12
e28c426d-cbef-4e2d-aa06-1ba0b6b1902f,11
6ddddc40-5b69-4253-8a26-167999a8bf96,10
aef30de1-c67e-4ab7-8577-2931a4bf3c7e,10


merchant_code       

In [16]:
df_anz.groupby(['merchant_code','bpay_biller_code'])['status'].agg(['count']).sort_values(by='count',ascending=0)
#There is 1 merchant_code '0.0' shows 883 times = bpay_biller_code of '0'

,,count
merchant_code,bpay_biller_code,
0.0,0,883


first_name          

In [17]:
#there are 100 first names = 100 accounts = 100 long_lat transactions in last 3 months

balance              

In [18]:
print ('min:', df_anz.balance.min())
print ('max:', df_anz.balance.max())

min: 0.24
max: 267128.52


In [19]:
anz_balance = df_anz.groupby(['account','first_name','long_lat','balance'])['status'].agg(['count'])#sort_values(by='balance',ascending=0)
#There are 12040 balances in 100 accounts
anz_balance = anz_balance.reset_index().sort_values(by='balance',ascending=0)
anz_balance

,account,first_name,long_lat,balance,count
4498,ACC-2014856841,Patrick,144.99 -37.90,267128.52,1
4497,ACC-2014856841,Patrick,144.99 -37.90,267093.66,1
4496,ACC-2014856841,Patrick,144.99 -37.90,267050.66,1
4495,ACC-2014856841,Patrick,144.99 -37.90,267028.97,1
4494,ACC-2014856841,Patrick,144.99 -37.90,266981.30,1
4493,ACC-2014856841,Patrick,144.99 -37.90,266916.32,1
4492,ACC-2014856841,Patrick,144.99 -37.90,266894.32,1
4491,ACC-2014856841,Patrick,144.99 -37.90,266877.26,1
4490,ACC-2014856841,Patrick,144.99 -37.90,266754.88,1
4489,ACC-2014856841,Patrick,144.99 -37.90,266704.88,1


date                 

In [20]:
print ('min:', df_anz.date.min())
print ('max:', df_anz.date.max())
#transactions during last 3 months from 1st Aug to 31st Oct

min: 2018-08-01 00:00:00
max: 2018-10-31 00:00:00


In [21]:
df_anz.groupby(['date'])['status'].agg(['count']).sort_values(by='count',ascending=0).head(10)
#Top 10 most crownded transaction days

,count
date,
2018-09-28,174
2018-08-17,172
2018-10-05,168
2018-10-17,162
2018-09-14,161
2018-09-21,160
2018-10-03,160
2018-09-27,159
2018-10-04,159


gender               

In [22]:
df_anz.gender.unique()

array(['F', 'M'], dtype=object)

In [23]:
df_anz.groupby(['gender'])['status'].agg(['count']).sort_values(by='count',ascending=0)
#Statistic of M and F are approximate

,count
gender,
M,6285
F,5758


age                  

In [24]:
print ('min:', df_anz.age.min())
print ('max:', df_anz.age.max())
#customers age from 18 to 78

min: 18
max: 78


In [25]:
df_anz.groupby(['gender'])['age'].agg(['min','max','count']).sort_values(by='count',ascending=0)
#Statistic of M and F are approximate

,min,max,count
gender,,,
M,18,78,6285
F,18,64,5758


In [26]:
df_anz.groupby(['age'])['status'].agg(['count']).sort_values(by='count',ascending=0).head(10)
#19-40 years old customers make the most transactions in ANZ bank

,count
age,
26,893
38,767
20,765
21,733
24,732
40,704
19,693
34,537
25,515


merchant_suburb      

In [27]:
df_anz.groupby(['merchant_suburb'])['status'].agg(['count']).sort_values(by='count',ascending=0).head(10)
#There are 1609 suburbs of customer transaction locations

,count
merchant_suburb,
Melbourne,255
Sydney,233
Southport,82
Brisbane City,79
Chatswood,55
Perth,45
Adelaide,44
Mount Gambier,41
Broadbeach,41


merchant_state       

In [28]:
df_anz.groupby(['merchant_state'])['status'].agg(['count']).sort_values(by='count',ascending=0)
#There are 7 states 

,count
merchant_state,
NSW,2169
VIC,2131
QLD,1556
WA,1100
SA,415
NT,205
ACT,73
TAS,68


In [29]:
df_anz.groupby(['merchant_state','merchant_suburb'])['status'].agg(['count']).sort_values(by='count',ascending=0).head(10)
#Top 10 transaction locations

count
merchant_state merchant_suburb       
VIC            Melbourne          255
NSW            Sydney             233
QLD            Southport           82
               Brisbane City       79
NSW            Chatswood           55
WA             Perth               45
SA             Adelaide            44
               Mount Gambier       41
QLD            Broadbeach          41
NSW            Parramatta          39

extraction           

In [30]:
print ('min:', df_anz.extraction.min())
print ('max:', df_anz.extraction.max())
#T01:01:15? = Time: 1:1:15am. T23:59:44? = Time: 23:59:44am

min: 2018-08-01T01:01:15.000+0000
max: 2018-10-31T23:59:44.000+0000


In [31]:
df_anz.groupby(['extraction'])['status'].agg(['count']).sort_values(by='count',ascending=0).head(10)
#There are 9442 extraction/withraw date and time
#Top 10 extraction/withdraw time shows 11am and 5pm are customer's most prefering period
# Cách lấy thời gian trong extraction ra 1 column riêng để phân tích thời gian yêu thích nhất của khách?

,count
extraction,
2018-09-03T11:00:00.000+0000,10
2018-08-07T17:00:00.000+0000,9
2018-10-02T17:00:00.000+0000,9
2018-08-28T17:00:00.000+0000,9
2018-10-19T13:00:00.000+0000,9
2018-10-01T11:00:00.000+0000,9
2018-09-04T17:00:00.000+0000,9
2018-10-23T17:00:00.000+0000,9
2018-09-17T11:00:00.000+0000,9


amount               

In [32]:
print ('min:', df_anz.amount.min())
print ('max:', df_anz.amount.max())

min: 0.1
max: 8835.98


In [33]:
df_anz.amount.describe()

count    12043.000000
mean       187.933588
std        592.599934
min          0.100000
25%         16.000000
50%         29.000000
75%         53.655000
max       8835.980000
Name: amount, dtype: float64

In [34]:
df_anz.groupby(['amount'])['status'].agg(['count']).sort_values(by='count',ascending=0).head()
# Cách làm tròn amount hay phân ra (0-2000, 2000-5000, 5000-8000) và tìm count để biết khoản chuyển thường nhất của khách? 
#Từ đó tính ra anual salary?

,count
amount,
33.0,168
39.0,119
29.0,116
36.0,107
45.0,102


transaction_id 

In [35]:
df_anz.groupby(['transaction_id'])['status'].agg(['count']).sort_values(by='count',ascending=0).head(1)
#12 047 transaction_id 

,count
transaction_id,
0001043c134f43bda3e3d87b2ee79c9c,1


country              

In [36]:
df_anz.country.unique()
#transactions are only made in Australia

array(['Australia'], dtype=object)

customer_id 

In [37]:
df_anz.groupby(['customer_id'])['status'].agg(['count']).sort_values(by='count',ascending=0).head()
#100 customer_id = 100 accounts = 100 first_name = 100 long_lat transactions in last 3 months

,count
customer_id,
CUS-2487424745,578
CUS-2142601169,303
CUS-3026014945,292
CUS-3378712515,260
CUS-1614226872,259


merchant_long_lat

In [38]:
df_anz.groupby(['merchant_long_lat'])['status'].agg(['count']).sort_values(by='count',ascending=0).head(10)
#there are 2703 merchant_long_lat during last 3 months transactions
#top 10 most merchant_long_lat during last 3 months

,count
merchant_long_lat,
151.21 -33.87,145
144.96 -37.82,85
144.97 -37.81,59
144.96 -37.81,56
153.02 -27.47,46
151.18 -33.8,45
151.2 -33.87,44
153.03 -27.47,36
151.21 -33.84,36


movement 

In [39]:
df_anz.movement.unique()

array(['debit', 'credit'], dtype=object)

In [40]:
df_anz.groupby(['movement'])['status'].agg(['count']).sort_values(by='count',ascending=0).head(10)
#Debit card is the most common used movement preference

,count
movement,
debit,11160
credit,883
